In [1511]:
from pyscipopt import Model, quicksum
import numpy as np
from itertools import product
import json

In [1512]:
#Importing constant variables
with open('./EVRP_LP/input_variables.json', 'r') as file:
    input_variables = json.load(file)

# print(input_variables["Qm"])

In [1513]:
def instanSacy(n_customers, n_vehicles, max_demand, max_distance):
    No = set(np.arange(1,n_customers+1)) #Set of customers
    N = No | {0} #Customers + depot
    F = {"F1", "F2"} #Set of charging stations
    NF = No | F #Customers + charging stations
    G = N | F #Customers, depot and charging stations
    A = [(i,j) for i,j in product(G,G) if i!=j] #Set of arcs between the nodes
    
    demand = {i: (0 if i in F | {0} else int(np.random.randint(100, max_demand, 1)[0])) for i in G} #Demand per customer

    M = list(np.arange(1,n_vehicles+1)) #Set of vehicles

    load_capacity = {m:input_variables["Qm"] for m in M} #Load_capacity per vehicle
    
    '''Time cost as a function of distance and avg. speed'''
    distance =  {(i,j):int(np.random.randint(50, max_distance, 25)[0]) for i,j in A}
    time_cost = {(i,j):round((distance[i,j] / input_variables["Avg_V"]),2) for i,j in A} #Travel time cost per arc
    
    battery_capacity = input_variables["Battery_capacity_Wh"]

    '''Energy cost as a function of Energy Consumption per Km'''
    EV_avg_energy_consumption = (290) #in W font:'https://ev-database.org/cheatsheet/energy-consumption-electric-car'
    CE_avg_energy_consumption = (0.16) #Liters/km font:'https://www.smmt.co.uk/wp-content/uploads/sites/2/Heavy-CV-Fuel-Consumption-Fact-Sheet.pdf'
    EV_energy_cost = {(i,j):(EV_avg_energy_consumption*distance[i,j]) for i,j in A} #Energy cost per arc
    CE_energy_cost = {(i,j):(CE_avg_energy_consumption*distance[i,j]) for i,j in A} #Fuel cost per arc
    # energy_cost = {(i,j):int(np.random.randint(1, max_energy_cost, 1)[0]) for i,j in A if j>i} #Energy cost per arc

    
    return No, N, F, NF, G, A, M, demand,load_capacity, battery_capacity, time_cost, distance, EV_energy_cost, CE_energy_cost

In [1514]:
def sacystation(No, N, F, NF, G, A, M, demand,load_capacity, battery_capacity, time_cost, distance, EV_energy_cost, CE_energy_cost):
    """transp -- model for solving the transportation problem
    Parameters:
        No - set of customers
        N - customers + depot
        F - set of charging stations
        G - N + F
        A - set of arcs (i,j)
        M - set of vehicles
        energy_cost[i,j] - energy transportation cost on arc (i,j)
        demand[i] - demand of customer i
        load_capacity[m] - capacity of vehicle m
        time_cost - Travel time per arc(i,j)

    ==================================
    Problems:
    veículo saindo direto para F;
    implementar tempo de recarga
    """
    
    model = Model("sacystation")

    #Binary decision variable
    x = {}
    for (i,j,m) in [(i,j,m) for i,j in A for m in M]:
        x[i,j,m] = model.addVar(vtype="B", name=f"x{(i,j,m)}")
        
    #Load Variable
    l = {}
    for (i,j,m) in [(i,j,m) for i,j in A for m in M]:
        l[i,j,m] = model.addVar(vtype="C", lb=0, ub=input_variables["Qm"], name=f"l{(i,j,m)}")
    
    #Remaining Battery Variable
    y = {}
    for (i, m) in [(i, m) for i in G for m in M]:
        y[i, m] = model.addVar(vtype="C", lb=0, ub=input_variables["Battery_capacity_Wh"],name=f"y[{i},{m}]")

    #Charge amount Variable
    v = {}
    for (i, m) in [(i, m) for i in F for m in M]:
        v[i, m] = model.addVar(vtype="C", lb=0, ub=input_variables["Battery_capacity_Wh"],name=f"v[{i},{m}]")

    """
    Restrições Estruturais
    """
    '''
    Routing constraints
    '''
    '''
    Each customer is visited only once
    '''        
    for i in No:
        model.addCons(quicksum(x[i,j,m] for j in G if i!=j for m in M ) == 1, name=f'customer_visiting[{i},{j},{m}]')
    
    '''
    Each charging stations is visited at most once
    '''
    for i in F:
        model.addCons(quicksum(x[i,j,m] for j in N for m in M) <= 1, name=f'charging_visiting[{i},{j},{m}]')
    
    '''
    Tour connectivity
    '''
    for m in M:
        for j in NF:
            model.addCons(quicksum(x[j,i,m] for i in G if i!=j) ==
                         quicksum(x[i,j,m] for i in G if i!=j)
                        , name=f'tour_connectivity[{i},{j},{m}]')
        
    '''
    Vehicles covers at most one tour
    ''' 
    for m in M:
        model.addCons(quicksum(x[0,j,m] for j in NF) <= 1, name=f'vehicle_one_tour[{0},{j},{m}]')                     
        
    '''
    Enforces that the vehicle starts from the depot and return to it
    after the tour
    '''    
    for m in M:
        model.addCons(quicksum(x[0, j, m] for j in No if j != 0) == 1,   name=f'departure_from_depot[{m}]') #Vehicle must leave the depot to visit customer (No)
        model.addCons(quicksum(x[j, 0, m] for j in G if j != 0) == 1,   name=f'return_to_depot[{m}]') #Can return to the depot from anywhere
        model.addCons(quicksum(x[0,j,m] for j in G if j!=0) ==
                      quicksum(x[j,0,m] for j in G if j!=0), name=f'depot_to_depot[{m}]')
            
    '''
    Time limit constraint
    '''
    for m in M:
        model.addCons(quicksum(time_cost[i,j]*x[i,j,m] for (i,j) in A if i!=j) <= 8, name=f'timeLimit[{m}]')
                
    # #==============================
    # # mostrar reunião
    
    '''
    Demand constraints
    '''        
    '''
    Demand Satisfaction
    '''
    for m in M:
        for i in G:
            if i in F or {0}:
                continue
            else:
                model.addCons((quicksum(l[k,i,m] for k in G if (k,i) in A) -
                          quicksum(l[i,j,m] for j in G if (i,j) in A))
                          == demand[i], name=f'demand_satisfaction[{i}]')   
        
    '''
    Vehicle Load Capacity
    ''' 
    for m in M:
        for (i,j) in A:
            model.addCons(demand[j] * x[i,j,m] <= l[i,j,m], name=f'load_capacity[{i},{j},{m}')     
            model.addCons(l[i,j,m] <= (input_variables["Qm"] - demand[i]) * x[i,j,m], name=f'load_capacity[{i},{j},{m}')

    """
    Battery Constraints
    """
    '''
    Battery available energy
    '''
    for i in No:
        for j in G:
            if i!=j:
                for m in M:
                    model.addCons(y[j,m] <= y[i,m] - EV_energy_cost[i,j] + input_variables["Battery_capacity_Wh"] * (1 - x[i,j,m]), name=f'battery_flow[{i},{j}]')
                    
    '''
    Battery available energy when leaving a charging station
    '''
    for i in F | {0}:#MAYBE I'LL HAVE TO IMPLEMENT THE DEPOT LATER
        for j in G - {0}:
            if i!=j:
                for m in M:
                    if i in F:
                        model.addCons(y[j,m] <= v[i,m] + y[i,m] - EV_energy_cost[i,j]*x[i,j,m] + input_variables["Battery_capacity_Wh"]*(1-x[i,j,m]), name=f'battery_flow[{i},{j}]')
                    else:  # If i is the depot
                        model.addCons(y[j, m] <= y[i, m] - EV_energy_cost[i, j] * x[i, j, m] + input_variables["Battery_capacity_Wh"] * (1 - x[i, j, m]), name=f'battery_flow_depot[{i},{j},{m}]')      

    '''
    Maximum recharge at charging stations
    '''
    for i in F:
        for m in M:
            model.addCons(v[i,m] + y[i,m] >= 0, name=f'max_recharge[{i},{m}]')
            model.addCons(v[i,m] + y[i,m] <= 0.8 * input_variables["Battery_capacity_Wh"], name=f'max_recharge[{i},{m}]')
            
    '''
    Initial charge at depot
    '''
    for m in M:
        model.addCons(y[0,m] == input_variables["ICp"] * input_variables["Battery_capacity_Wh"] * quicksum(x[0,j,m] for j in G if j!=0), name=f'depot_charge[{m}]')
        
    '''
    Positive values for energy levels
    '''
    for j in G - {0}:
        for m in M:
            model.addCons(y[j,m] <= (1 / input_variables['ICp']) * y[0,m], name=f'positive_energy_level[{j}]')
    
    
    # Objective - ONLY TRAVEL TIME and WAITING TIME 
    travel_time_cost = quicksum(input_variables["Ctt"]*time_cost[i,j]*x[i,j,m] for (i,j) in A for m in M)
    model.setObjective(travel_time_cost, "minimize")

    model.optimize()

    model.data = x
    
    return model, y


In [1515]:
if __name__ == "__main__":
    No, N, F, NF, G, A, M, demand,load_capacity, battery_capacity, time_cost, distance, EV_energy_cost, CE_energy_cost = instanSacy(3,2,8000,100)
    model, y = sacystation(No, N, F, NF, G, A, M, demand,load_capacity, battery_capacity, time_cost, distance, EV_energy_cost, CE_energy_cost)    
    model.optimize()
    
    EPS = 1.e-6  # a small tolerance
    x = model.data  # model.data contains the decision variables

    # Time cost
    # cost = sum(time_cost[i,j]*x[i,j,m] for i,j in A if i!=j)
    # values = cost.terms.values()
    # print("Total time:", sum(values))
    # print(f'Travel time ={(travel_time_cost.terms.values())}')
    # print(f'{time_cost=}')
    # print(f'Total time cost={sum(time_cost.values())}')
    # print(f'{load_capacity=}')
    # print(f'{demand=}')
    
    if model.getStatus() == "optimal":
        print("Optimal value:", model.getObjVal())
    else:
        print("Problem could not be solved to optimality")      

    total_distance = sum(distance[i, j] for i, j, m in [(i, j, m) for i, j in A for m in M] if model.getVal(x[i, j, m]) > 0.5)
    print("Total distance traveled:", total_distance)

    total_energy_cost = sum(EV_energy_cost[i, j] for i, j, m in [(i, j, m) for i, j in A for m in M] if model.getVal(x[i, j, m]) > 0.5)
    print("Total energy cost:", total_energy_cost)
        
    # print(f'{y.items()}')
    print(f'{EV_energy_cost.items()}')
    
    for (i, j, m) in x:
        if model.getVal(x[i, j, m]) > 0.5:
            print(f"x({i}, {j}, {m}) = {model.getVal(x[i, j, m])}")
    #     # else:
    #     #     print(f"x[{i}, {j}, {m}] = 0")
    


Optimal value: 103.52000000000002
Total distance traveled: 518
Total energy cost: 150220
dict_items([((0, 1), 25520), ((0, 2), 15950), ((0, 3), 22040), ((0, 'F2'), 19140), ((0, 'F1'), 25230), ((1, 0), 26970), ((1, 2), 28130), ((1, 3), 27550), ((1, 'F2'), 28710), ((1, 'F1'), 28130), ((2, 0), 26680), ((2, 1), 19430), ((2, 3), 28420), ((2, 'F2'), 25810), ((2, 'F1'), 25810), ((3, 0), 21460), ((3, 1), 17400), ((3, 2), 21170), ((3, 'F2'), 28710), ((3, 'F1'), 28130), (('F2', 0), 15950), (('F2', 1), 25810), (('F2', 2), 19720), (('F2', 3), 20010), (('F2', 'F1'), 26970), (('F1', 0), 24940), (('F1', 1), 15370), (('F1', 2), 20590), (('F1', 3), 22040), (('F1', 'F2'), 18850)])
x(0, 2, 2) = 1.0
x(0, 3, 1) = 1.0
x(1, F1, 1) = 1.0
x(2, F2, 2) = 1.0
x(3, 1, 1) = 1.0
x(F2, 0, 2) = 1.0
x(F1, 0, 1) = 0.9999999999999999


In [1516]:
model.writeProblem(r"D:\DAY2DAY\MESTRADO\Codes\SCIP\LPs\problem.lp")
model.freeProb()

wrote problem to file D:\DAY2DAY\MESTRADO\Codes\SCIP\LPs\problem.lp


In [1517]:
model = Model("sacystation")

def instanSacy(n_customers, n_vehicles, max_distance):
    No = set(np.arange(1,n_customers+1)) #Set of customers
    N = No | {0}
    F = {0, "F1", "F2"} #Set of recharging stations
    G = N | F
    A = [(i,j) for i,j in product(G,G) if i!=j]
    
    for i in G:     
        demand = {i: (0 if i in F else int(np.random.randint(100, 300, 1)[0])) for i in G} #Demand per customer
        
    M = list(np.arange(1,n_vehicles+1)) #Set of vehicles

    load_capacity = {m:input_variables["Qm"] for m in M} #Load_capacity per vehicle
    
    '''Time cost as a function of distance and avg. speed'''
    distance =  {(i,j):int(np.random.randint(50, max_distance, 25)[0]) for i,j in A}
    time_cost = {(i,j):round((distance[i,j] / input_variables["Avg_V"]),2) for i,j in A} #Travel time cost per arc
    # time_cost = {(i,j):int(np.random.randint(1, max_time_cost, 1)[0]) for i,j in A} 
    
    battery_capacity = input_variables["Battery_capacity_Wh"]

    '''Energy cost as a function of Energy Consumption per Km'''
    EV_avg_energy_consumption = (290+150)/2 #in Wh font:'https://ev-database.org/cheatsheet/energy-consumption-electric-car'
    CE_avg_energy_consumption = (0.16) #Liters/km font:'https://www.smmt.co.uk/wp-content/uploads/sites/2/Heavy-CV-Fuel-Consumption-Fact-Sheet.pdf'
    EV_energy_cost = {(i,j):(EV_avg_energy_consumption/distance[i,j]) for i,j in A} #Energy cost per arc
    CE_energy_cost = {(i,j):(CE_avg_energy_consumption/distance[i,j]) for i,j in A} #Fuel cost per arc
    # energy_cost = {(i,j):int(np.random.randint(1, max_energy_cost, 1)[0]) for i,j in A if j>i} #Energy cost per arc

    
    return No, N, F, G, A, M, demand,load_capacity, battery_capacity, time_cost, distance, EV_energy_cost, CE_energy_cost

No, N, F, G, A, M, demand,load_capacity, battery_capacity, time_cost, distance, EV_energy_cost, CE_energy_cost = instanSacy(2,1,100)

model = Model("sacystation")

#Creating binary decision variable
x = {}
for (i,j,m) in [(i,j,m) for i,j in A for m in M ]:
    x[i,j,m] = model.addVar(vtype="B", name=f"x{(i,j,m)}")
    
L = {}
for (i,j,m) in [(i,j,m) for i,j in A for m in M]:
    L[i,j,m] = model.addVar(vtype="C", name=f"L{(i,j,m)}")    
    
y = {}
for (i,j,m) in [(i,j,m) for i,j in A for m in M]:
    y[i,j,m] = model.addVar(vtype="C", name=f"y{(i,j,m)}")
    
# y2 = {}
# if i!=j:
#     for i, j, m in product(A,M) :
#         y2[i, j, m] = model.addVar(vtype="C", name=f"y2({i},{j},{m})")
 
# print(len([(i,j,m) for i,j,m in product(G,G,M) if i!=j]))    
print(G)    
# print(len(A), y, len(y), sep='\n')

x2 = {}
for (i,j) in A:
    for m in M:
        x2[i,j,m] = model.addVar(vtype="B", name=f"x{(i,j,m)}")
print(len(x),len(x2))
print(No)





{0, 1, 2, 'F2', 'F1'}
20 20
{1, 2}
